In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import time
import statsmodels.api as sm
from sklearn import linear_model
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error
from statsmodels.regression.quantile_regression import QuantReg

from sklearn.preprocessing import StandardScaler

import os

from sklearn.model_selection import train_test_split

In [82]:
path = 'data'+os.path.sep
df = pd.read_csv(path+'market_info_soul.csv',encoding='utf-8',delimiter='|')

In [83]:
len(df)

367534

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367534 entries, 0 to 367533
Data columns (total 39 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   상가업소번호     367534 non-null  int64  
 1   상호명        367534 non-null  object 
 2   지점명        56770 non-null   object 
 3   상권업종대분류코드  367534 non-null  object 
 4   상권업종대분류명   367534 non-null  object 
 5   상권업종중분류코드  367534 non-null  object 
 6   상권업종중분류명   367534 non-null  object 
 7   상권업종소분류코드  367534 non-null  object 
 8   상권업종소분류명   367534 non-null  object 
 9   표준산업분류코드   344363 non-null  object 
 10  표준산업분류명    344363 non-null  object 
 11  시도코드       367534 non-null  int64  
 12  시도명        367534 non-null  object 
 13  시군구코드      367534 non-null  int64  
 14  시군구명       367534 non-null  object 
 15  행정동코드      367534 non-null  int64  
 16  행정동명       367458 non-null  object 
 17  법정동코드      367534 non-null  int64  
 18  법정동명       367534 non-null  object 
 19  지번코드       367534 non-n

In [84]:
df

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,지번코드,대지구분코드,대지구분명,지번본번지,지번부번지,지번주소,도로명코드,도로명,건물본번지,건물부번지,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,19911397,커피빈코리아선릉로93길점,코리아선릉로93길점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,비알콜 음료점업,11,서울특별시,11680,강남구,1168064000,역삼1동,1168010100,역삼동,1168010100206960042,1,대지,696,42.0,서울특별시 강남구 역삼동 696-42,116804166502,서울특별시 강남구 선릉로93길,6,NaN,1168010100106960042022041,NaN,서울특별시 강남구 선릉로93길 6,135080,6149.0,NaN,1,NaN,127.047883,37.505675
1,19911801,프로포즈,NaN,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,기타 주점업,11,서울특별시,11545,금천구,1154563000,독산3동,1154510200,독산동,1154510200201620001,1,대지,162,1.0,서울특별시 금천구 독산동 162-1,115453117003,서울특별시 금천구 가산로,34,6.0,1154510200101620001017748,NaN,서울특별시 금천구 가산로 34-6,153010,8545.0,NaN,1,NaN,126.899220,37.471711
2,19905471,와라와라호프,NaN,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,기타 주점업,11,서울특별시,11680,강남구,1168060000,대치1동,1168010600,대치동,1168010600206040000,1,대지,604,NaN,서울특별시 강남구 대치동 604,116802000003,서울특별시 강남구 남부순환로,2933,NaN,1168010600106040000014378,대치상가,서울특별시 강남구 남부순환로 2933,135280,6280.0,NaN,1,NaN,127.061026,37.493922
3,19932756,가락사우나내스낵,NaN,F,생활서비스,F09,대중목욕탕/휴게,F09A02,사우나/증기탕/온천,S96121,욕탕업,11,서울특별시,11710,송파구,1171060000,석촌동,1171010500,석촌동,1171010500202560000,1,대지,256,NaN,서울특별시 송파구 석촌동 256,117103123001,서울특별시 송파구 가락로,71,NaN,1171010500102560005010490,NaN,서울특별시 송파구 가락로 71,138846,5690.0,NaN,1,NaN,127.104071,37.500249
4,19993199,고려대학교교육관쎄리오점,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,비알콜 음료점업,11,서울특별시,11290,성북구,1129060000,안암동,1129012500,안암동5가,1129012500200010002,1,대지,1,2.0,서울특별시 성북구 안암동5가 1-2,112903005035,서울특별시 성북구 안암로,145,NaN,1129012500100010002039525,고려대학교안암캠퍼스(인문사회계),서울특별시 성북구 안암로 145,136701,2841.0,NaN,1,32,127.031702,37.588485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367529,16768274,혼밥대왕,용문점,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,한식 음식점업,11,서울특별시,11170,용산구,1117059000,용문동,1117012000,도원동,1117012000200230000,1,대지,23,NaN,서울특별시 용산구 도원동 23,111703005018,서울특별시 용산구 새창로,70,NaN,1117012000100230000015904,도원동삼성래미안,서울특별시 용산구 새창로 70,140748,4354.0,NaN,NaN,NaN,126.954928,37.538597
367530,16768252,쭈대가제육대가,금천점,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,한식 음식점업,11,서울특별시,11545,금천구,1154561000,독산1동,1154510200,독산동,1154510200201440043,1,대지,144,43.0,서울특별시 금천구 독산동 144-43,115454151282,서울특별시 금천구 시흥대로151길,32,NaN,1154510200101440043018212,성준리치빌,서울특별시 금천구 시흥대로151길 32,153806,8531.0,NaN,NaN,NaN,126.896774,37.475878
367531,16768239,더볶,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,한식 음식점업,11,서울특별시,11620,관악구,1162066500,서림동,1162010200,신림동,1162010200201050079,1,대지,105,79.0,서울특별시 관악구 신림동 105-79,116204160456,서울특별시 관악구 서림길,52,NaN,1162010200101050079018220,NaN,서울특별시 관악구 서림길 52,151855,8837.0,NaN,NaN,NaN,126.938386,37.473800
367532,16775207,도가횟집,NaN,Q,음식,Q03,일식/수산물,Q03A01,횟집,I56111,한식 음식점업,11,서울특별시,11680,강남구,1168054500,압구정동,1168010700,신사동,1168010700205890001,1,대지,589,1.0,서울특별시 강남구 신사동 589-1,116804166272,서울특별시 강남구 도산대로33길,18,NaN,1168010700105890001026475,골든빌,서울특별시 강남구 도산대로33길 18,135893,6025.0,NaN,NaN,NaN,127.029974,37.521757


In [20]:
pd.set_option('display.max_rows',500)

In [10]:
pd.set_option('display.max_columns',100)

In [85]:
df = df[['상호명','상가업소번호','지점명','상권업종대분류명','상권업종중분류명','상권업종소분류명','표준산업분류명','시도명','법정동명','경도','위도']]

In [30]:
df['상권업종대분류명'].unique()

array(['음식', '생활서비스', '소매', '관광/여가/오락', '학문/교육', '부동산', '숙박', '스포츠'],
      dtype=object)

In [31]:
df[df['상권업종대분류명']=='학문/교육']['상권업종중분류명'].unique()

array(['학원-어학', '학원-음악미술무용', '학원-보습교습입시', '학원-예능취미체육', '유아교육', '학원기타',
       '학문교육기타', '학원-자격/국가고시', '학원-컴퓨터', '도서관/독서실', '학원-창업취업취미'],
      dtype=object)

In [52]:
df[df['상권업종대분류명']=='학문/교육'].groupby(['상권업종중분류명','상권업종소분류명']).count()

상호명  상가업소번호  지점명  상권업종대분류명  표준산업분류명   시도명  행정동명  \
상권업종중분류명   상권업종소분류명                                                        
도서관/독서실    고시원           853     853   47       853      853   853   853   
           독서실           914     914   56       914      914   914   914   
           이동도서관운영        18      18    1        18       18    18    18   
유아교육       놀이방           116     116    1       116      116   116   116   
           동화구연            2       2    0         2        2     2     2   
           블럭놀이방          17      17    1        17       17    17    17   
           실내놀이터-유아       71      71   11        71       71    71    71   
           어린이집         3700    3700    7      3700     3700  3700  3700   
           유아교육기관기타       90      90    7        90       90    90    90   
학문교육기타     시험정보안내          5       5    0         5        5     5     5   
           운동/코치학교        53      53    4        53       53    53    53   
           학습지보급         424     424   77       424      424   424   424   
학원-보습교습입시  학원-검정고시         7       7    3         7        7     7     7   
           학원-입시        6706    6706  656      6706     6706  6706  6706   
           학원-편입          24      24   10        24       24    24    24   
학원-어학      학원-어린이영어      306     306   81       306      306   306   306   
           학원-외국어/어학    3269    3269  430      3269     3269  3269  3268   
학원-예능취미체육  검도도장           97      97    1        97       97    97    97   
           레크레이션지도/상담      3       3    0         3        3     3     3   
           유도장            46      46    2        46       46    46    46   
           태권도장          640     640    5       640      640   640   640   
           학원-가요          35      35    3        35       35    35    35   
           학원-기억법          4       4    0         4        4     4     4   
           학원-꽃꽃이         25      25    5        25       25    25    25   
           학원-레크리에이션       1       1    0         1        1     1     1   
           학원-마술           6       6    0         6        6     6     6   
           학원-바둑          34      34    2        34       34    34    34   
           학원-배우/탤런트      84      84   18        84       84    84    84   
           학원-사교춤         46      46   11        46       46    46    46   
           학원-사진           2       2    1         2        2     2     2   
           학원-서당          18      18    2        18       18    18    18   
           학원-스포츠/재즈댄스   244     244   23       244      244   244   244   
           학원-아트플라워        1       1    0         1        1     1     1   
           학원-에어로빅        23      23    2        23       23    23    23   
           학원-요리          67      67   18        67       67    67    67   
           학원-웅변          30      30    7        30       30    30    30   
           학원-자수/동양자수      3       3    0         3        3     3     3   
           학원-주산암산부기속셈   113     113   25       113      113   113   113   
           학원-지점토          1       1    0         1        1     1     1   
           학원-펜글씨/한문      24      24    1        24       24    24    24   
           합기도장          104     104    1       104      104   104   104   
학원-음악미술무용  서예/서화/미술     1473    1473  228      1473     1473  1473  1473   
           피아노/바이올린/기타  2756    2756  341      2756     2756  2756  2755   
           학원-국악          43      43    2        43       43    43    43   
           학원-무용         214     214   37       214      214   214   214   
학원-자격/국가고시 학원-간호조무사       71      71   11        71       71    71    71   
           학원-건축토목기술       7       7    2         7        7     7     7   
           학원-경찰           4       4    3         4        4     4     4   
           학원-공인중개사       17      17    6        17       17    17    17   
           학원-관광통역         2       2    0         2        2     2     2   
           학원-국가고시         7       7    1         7        7     7     7 

### 대 : '학문/교육'

In [86]:
df = df[df['상권업종대분류명']=='학문/교육']

### 중 : 학문교육기타 ,학원-보습교습입시 ,학원-어학

In [63]:
df

,상호명,상가업소번호,지점명,상권업종대분류명,상권업종중분류명,상권업종소분류명,표준산업분류명,시도명,행정동명,경도,위도
66,안재형영어,20736806,NaN,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,목5동,126.875973,37.531592
80,정아트,20769711,NaN,학문/교육,학원-음악미술무용,서예/서화/미술,예술 학원,서울특별시,염창동,126.873645,37.550924
82,삼익피아노강동대리점,20927403,강동대리점,학문/교육,학원-음악미술무용,피아노/바이올린/기타,예술 학원,서울특별시,천호3동,127.129075,37.539262
83,설악피아노학원,20966240,NaN,학문/교육,학원-음악미술무용,피아노/바이올린/기타,예술 학원,서울특별시,잠원동,127.015432,37.517844
89,이엠학원,20861438,NaN,학문/교육,학원-보습교습입시,학원-입시,일반 교과 학원,서울특별시,군자동,127.074812,37.554303
...,...,...,...,...,...,...,...,...,...,...,...
367464,클랑미술교습소,16719507,NaN,학문/교육,학원-음악미술무용,서예/서화/미술,예술 학원,서울특별시,고덕2동,127.162422,37.560516
367465,아트앤하트삼익그린미술교습소,16719512,NaN,학문/교육,학원-음악미술무용,서예/서화/미술,예술 학원,서울특별시,명일1동,127.146260,37.551462
367466,학원,16719314,윤선생IGSE영재,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,석관동,127.068979,37.613753
367467,황명주중국어랩,16719291,NaN,학문/교육,학원-어학,학원-외국어/어학,외국어학원,서울특별시,청림동,126.960977,37.491589


In [87]:
df = df[df['상권업종중분류명'].apply(lambda x : x in ['학문교육기타','학원-보습교습입시','학원-어학']) ]

### 소 : '학원-외국어/어학','학원-입시'

In [67]:
df.groupby(['상권업종소분류명']).count()

,상호명,상가업소번호,지점명,상권업종대분류명,상권업종중분류명,표준산업분류명,시도명,행정동명,경도,위도
상권업종소분류명,,,,,,,,,,
시험정보안내,5,5,0,5,5,5,5,5,5,5
운동/코치학교,53,53,4,53,53,53,53,53,53,53
학습지보급,424,424,77,424,424,424,424,424,424,424
학원-검정고시,7,7,3,7,7,7,7,7,7,7
학원-어린이영어,306,306,81,306,306,306,306,306,306,306
학원-외국어/어학,3269,3269,430,3269,3269,3269,3269,3268,3269,3269
학원-입시,6706,6706,656,6706,6706,6706,6706,6706,6706,6706
학원-편입,24,24,10,24,24,24,24,24,24,24


In [88]:
df = df[df['상권업종소분류명'].apply(lambda x : x in ['학원-외국어/어학','학원-입시'] ) ]

### 상권업종소분류명  : '학원-외국어/어학','학원-입시' 가능한 범주

In [89]:
len(df)

9975

#### 학원-외국어/어학 : 3269
#### 학원-입시        : 6706


In [90]:
udf = df[df['상권업종소분류명']=='학원-입시'] 

In [91]:
edf = df[df['상권업종소분류명']=='학원-외국어/어학'] 

In [93]:
len(udf),len(edf)

(6706, 3269)

In [97]:
selected = ['상호명','상권업종소분류명','경도','위도','법정동명']
udfc = udf[selected]
edfc = edf[selected]

#### 학원-외국어/어학 

In [107]:
len(edfc[edfc['상호명'].str.contains('중국') ]) # 중국어 이름 포함된 학원 116

116

In [113]:
edfc

,상호명,상권업종소분류명,경도,위도,법정동명
66,안재형영어,학원-외국어/어학,126.875973,37.531592,목동
135,지엔비화곡외국어학원,학원-외국어/어학,126.845150,37.545693,화곡동
150,김찬영어전문학원,학원-외국어/어학,126.848757,37.550306,화곡동
314,잉글리쉬무무,학원-외국어/어학,127.070939,37.550911,군자동
780,천천중국어,학원-외국어/어학,126.877967,37.535954,목동
...,...,...,...,...,...
367306,중국어덕후,학원-외국어/어학,127.069202,37.611349,석관동
367459,학원,학원-외국어/어학,126.929609,37.557986,창천동
367460,해법중국어원효점교습소,학원-외국어/어학,126.950049,37.533851,산천동
367466,학원,학원-외국어/어학,127.068979,37.613753,석관동


In [118]:
edfc.drop(index = edfc.loc[edfc['상호명'].str.contains('중국')].index,inplace = True)

C:\Users\epsilon\dev\Anaconda\envs\fortf\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [129]:
edfc.drop(index = edfc.loc[edfc['상호명'].str.contains('논술')].index,inplace=True)

C:\Users\epsilon\dev\Anaconda\envs\fortf\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#### 데이터 파다 발견 점프셈교실 이라는 학원이 서울에 200 개...ㄷㄷ
#### 점프셈교실 : 알라들 데리고 수학 가르치는 눈높이가는 곳

In [143]:
len(edfc[edfc.상호명 == '점프셈교실'])

200

In [146]:
edfc.drop(index = edfc[edfc.상호명 == '점프셈교실'].index,inplace=True)

C:\Users\epsilon\dev\Anaconda\envs\fortf\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [158]:
edfc.drop(index = edfc[edfc['상호명'].str.contains('(일본)|(한자)|(치일드)|(어린이)')].index, 

C:\Users\epsilon\dev\Anaconda\envs\fortf\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,상호명,상권업종소분류명,경도,위도,법정동명
894,목동ECC어린이영어교실,학원-외국어/어학,126.873267,37.522673,신정동
2101,진주원일본어교육학원,학원-외국어/어학,127.079930,37.647015,중계동
3775,용두어린이영어도서관,학원-외국어/어학,127.030300,37.576102,용두동
7225,형성한자,학원-외국어/어학,126.879496,37.535851,목동
10888,강남일본어문화원,학원-외국어/어학,127.028185,37.499189,역삼동
11600,일본어교육교류추진연구소,학원-외국어/어학,126.984104,37.569064,관철동
15491,시사일본어학원신촌캠퍼스,학원-외국어/어학,126.940293,37.558821,창천동
22563,주한영국문화원어린이어학원,학원-외국어/어학,126.870342,37.526078,목동
65490,제이에듀일본어연구소,학원-외국어/어학,126.870735,37.527064,목동
77054,시사일본어사,학원-외국어/어학,127.030075,37.496520,역삼동
